In [1]:
import tensorflow as tf

from deepface.DeepFace import build_model
from deepface.commons import functions

import lpips_tf

2024-02-13 18:06:35.415629: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 18:06:35.453043: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 18:06:35.453064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 18:06:35.453988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 18:06:35.460198: I tensorflow/core/platform/cpu_feature_guar

## Inputs

### Victim Models (F)

In [2]:
victim_models_labels = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "SFace"
]

F = set()

for model_name in victim_models_labels:
  F.add((build_model(model_name=model_name).model, functions.find_target_size(model_name)))
  
print(len(F))

2024-02-13 18:06:37.607116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 18:06:37.642348: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 18:06:37.643751: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

8


### HyperParams

In [3]:
lda = tf.Variable(0.2, trainable=False, name='lambda')

## Loss

### Embeddings Loss
$$
loss \leftarrow \dfrac{1}{\left\|\mathbb{F}\right\|} \sum^{\mathbb{F}}_{f} - \dfrac{f(x) \cdot f(x_{adv})} {\left\| f(x)\right\|_{2}\left\| f(x_{adv})\right\|_{2}}
$$

In [4]:
def f_cosine_similarity(x, x_adv, f):
  emb_t = f(x)
  emb_adv = f(x_adv)
  dist = tf.keras.losses.cosine_similarity(emb_t, emb_adv, axis=1)
  dist = tf.negative(dist)
  return dist

In [5]:
def F_loss(x, x_adv, loss, F):
  N = len(F)
  for f in F:
    model = f[0]
    in_shape = f[1]
    #TODO: convert to right shape
    loss = tf.add(loss, f_cosine_similarity(model, x, x_adv))
  loss = tf.divide(loss, N)
  return loss

### Perceptual Loss ($L_{pips}$)
$$
loss \leftarrow loss + \lambda L_{pips}(x_{adv}, x)
$$

In [6]:
def lpips_loss(x, x_adv, loss, lda):
  dist = lpips_tf.lpips(x_adv, x, model='net-lin', net='alex')
  dist = tf.multiply(lda, dist)
  loss = tf.add(loss, dist)
  return loss

### Loss Function

In [7]:
def reface_loss(x, x_adv, F, lda):
  floss = F_loss(x, x_adv, F)
  lpipsloss = lpips_loss(x, x_adv, lda)
  return tf.add(floss, lpipsloss)

## Loss Object
How to use this particular loss function in training.

In [8]:
from reface import ReFaceLoss, FaceEmbedEnum

In [9]:
F = [FaceEmbedEnum.DEEPFACE, FaceEmbedEnum.ARCFACE]

In [10]:
loss = ReFaceLoss(F, l=0.2)

In [11]:
loss.get_config()

{'name': 'ReFaceLoss',
 'F': [<FaceEmbedEnum.DEEPFACE: 'DeepFace'>,
  <FaceEmbedEnum.ARCFACE: 'ArcFace'>],
 'lambda': 0.2,
 'reduction': 'auto'}